In [30]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
cap = cv2.VideoCapture("12.avi")

In [32]:
while not cap.isOpened():
    cap = cv2.VideoCapture("12.avi")
    cv2.waitKey(1000)
    print("Wait for the header")

In [33]:
pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
fps = cap.get(cv2.CAP_PROP_FPS)

In [34]:
# https://www.pyimagesearch.com/2015/03/09/capturing-mouse-click-events-with-python-and-opencv/
refPt = []
cropping = False
title = "press 'n' for next, 'r' to reset, 'c' to stop"

In [35]:
def click_and_crop(event, x, y, flags, param):
    # grab references to the global variables
    global refPt, cropping
    
    # if the left mouse button is pressed, record the starting (x, y)
    # coordinates and indicate that cropping is being performed
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True
    
    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates and indicate that
        # the cropping operation is finished
        refPt.append((x, y))
        cropping = False
        
        # draw a rectangle around the region of interest
        cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow(title , image)

In [36]:
# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True, help="Path to the image")
# args = vars(ap.parse_args())
 
# load the image, clone it, and setup the mouse callback function
flag, image = cap.read() 
clone = image.copy()
cv2.namedWindow(title)
cv2.setMouseCallback(title, click_and_crop)
 
# keep looping until the 'q' key is pressed
while True:

    # display the image and wait for a keypress
    cv2.imshow(title, image)
    key = cv2.waitKey(1) & 0xFF
 
    # if the 'n' key is pressed, give a frame after 1 sec
    # for reselecting ROI
    if key == ord("n"):
        pos_frame = pos_frame + fps
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame)
        flag, image = cap.read()
        clone = image.copy()
        cv2.namedWindow(title)
        cv2.setMouseCallback(title, click_and_crop)
    
    
    # if the 'r' key is pressed, reset the cropping region
    if key == ord("r"):
        image = clone.copy()
 
    # if the 'c' key is pressed, break from the loop
    elif key == ord("c"):
        break
 
# if there are two reference points, then crop the region of interest
# from teh image and display it
if len(refPt) == 2:
    roi = clone[refPt[0][1]:refPt[1][1], refPt[0][0]:refPt[1][0]]
    cv2.imshow("ROI", roi)
    cv2.waitKey(0)
 
# close all open windows
cv2.destroyAllWindows()

In [37]:
ref_frame_pos = pos_frame

In [9]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [10]:
M = cv2.moments(image_gray)

In [11]:
cX = int(M["m10"] / M["m00"])
cY = int(M["m01"] / M["m00"])

In [12]:
while True:
    flag, frame = cap.read()
    if flag:
        pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        M = cv2.moments(frame_gray)
        cX_new = int(M["m10"] / M["m00"])
        cY_new = int(M["m01"] / M["m00"])
        newROI = [(refPt[0][0] + cX - cX_new, refPt[0][1] + cY - cY_new)];
        newROI.append(((refPt[1][0] + cX - cX_new), (refPt[1][1] + cY - cY_new)))
        cv2.rectangle(frame, newROI[0], newROI[1], (0, 0, 255), 2)
        cv2.imshow("ROI Tracking", frame)
    else:
        # The next frame is not ready, so we try to read it again
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame-1)
        print("frame is not ready")
        # Waiting 100ms
        cv2.waitKey(100)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        # If the number of captured frames is equal to the total number of frames,
        # we stop
        break
# close all open windows
cv2.destroyAllWindows()

In [40]:
cap.set(cv2.CAP_PROP_POS_FRAMES, ref_frame_pos)

True

In [41]:
flag, frame = cap.read()

In [42]:
cv2.imwrite("another.jpg", frame)

True